In [ ]:
import subprocess
import sys

COLAB = "google.colab" in sys.modules


def _install(package):
    if COLAB:
        ans = input(f"Install { package }? [y/n]:")
        if ans.lower() in ["y", "yes"]:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "--quiet", package]
            )
            print(f"{ package } installed!")


def _colab_install_missing_deps(deps):
    import importlib

    for dep in deps:
        if importlib.util.find_spec(dep) is None:
            _install(dep)


deps = ["ioos-metrics"]
_colab_install_missing_deps(deps)

In [ ]:
# The date format is YYYY-MM-DDTHH:mm:ss
query = {
    "min_time": "2025-01-01T00:00:00",
    "max_time": "2025-02-28T23:59:59",
    "min_lat": 10,
    "max_lat": 42,
    "min_lon": -99,
    "max_lon": -50,
}

In [ ]:
from ioos_metrics.ioos_metrics import _ngdac_gliders as ngdac_gliders


import pandas as pd

metadata_metrics = ngdac_gliders(**query)
metadata_metrics["time_coverage_start"] = pd.to_datetime(
    metadata_metrics["time_coverage_start"],
)
metadata_metrics["time_coverage_end"] = pd.to_datetime(
    metadata_metrics["time_coverage_end"],
)

_save_df = (
    metadata_metrics[
        [
            "glider",
            "wmo_id",
            "sea_name",
            "time_coverage_start",
            "time_coverage_end",
            "days",
            "num_profiles",
            "institution",
            "acknowledgment",
        ]
    ]
    .reset_index(
        names=["dataset_id"],
    )
    .sort_values(
        by="dataset_id",
        key=lambda x: x.str.lower(),
    )
    .rename(
        columns={
            "time_coverage_start": "start_date",
            "time_coverage_end": "end_date",
            "sea_name": "deployment_area",
        },
    )
)

_save_df["start_date"] = _save_df["start_date"].dt.strftime("%m-%d-%Y")
_save_df["end_date"] = _save_df["end_date"].dt.strftime("%m-%d-%Y")
_save_df["days"] = [d.days for d in _save_df["days"]]

_save_df.to_csv(
    path_or_buf="glider_metadata_metrics.csv",
    index=False,
)

## GeoDataFrame

In [ ]:
import geopandas


gdf = metadata_metrics.copy()

gdf = gdf.reset_index(names="glider deployment")
gdf = gdf.rename({"track": "geometry"}, axis=1)
gdf = geopandas.GeoDataFrame(gdf)
gdf = gdf.set_crs(epsg=4326)

In [ ]:
import folium

m = folium.Map()
for _, glider in gdf.iterrows():
    folium.GeoJson(
        data=glider["geometry"],
        tooltip=glider["glider deployment"],
        popup=f"{glider['time_coverage_start']} -- {glider['time_coverage_end']}",
    ).add_to(m)


folium.Rectangle(
    bounds=[
        (query["min_lat"], query["min_lon"]),
        (query["max_lat"], query["max_lon"]),
    ],
).add_to(m)
m.fit_bounds(m.get_bounds())

m

In [ ]:
print(f'Glider days: {metadata_metrics["days"].sum().days}')
print(f'Glider deployments: {len(metadata_metrics)}')
print(f'Number of profiles days: {metadata_metrics["num_profiles"].sum()}')